In [1]:
:opt no-lint
{-# LANGUAGE TypeSynonymInstances FlexibleInstances #-}

이제 그림 \ref{fig:bigStepCBV}와 \ref{fig:bigStepCBN}의 큰걸음 의미구조를 하스켈로 옮겨 보자.

In [2]:
type Nm = String -- 변수 이름은 문자열로
data Expr = Var Nm         -- x
          | Lam Nm Expr    -- (λx.e)
          | App Expr Expr  -- (e1 e2)
          deriving (Eq, Ord)

In [3]:
instance Show Expr where -- LaTeX 소스코드 생성하는 Show 인스턴스 직접 선언
  showsPrec _ (Var x) = showString x
  showsPrec p (Lam x e) = showParen (p > 1) $
      showString ("\\lambda "++x++".") . showsPrec 1 e
  showsPrec p (App e1 e2) = showParen (p > 9) $
      showsPrec 9 e1 . showString "\\;" . showsPrec 10 e2

import IHaskell.Display (latex)
instance IHaskellDisplay Expr where -- 노트북에 디스플레이하는 인스턴스 선언
    display e = display [latex $ "$"++show e++"$"]

In [4]:
Lam "x" (Lam "y" (App (Var "x") (Lam "z"  (Var "y"))))

In [5]:
-- 연산자 (.), ($) 및 역따옴표 중위 표현 활용 
Lam "x" . Lam "y" $ Var "x" `App` Lam "z" (Var "y")

In [6]:
type Env = [(Nm,Clos)] -- 값계산 환경
data Clos = Cl Expr Env deriving (Eq,Ord) -- 클로저

In [7]:
import Data.List (intercalate)
instance {-# OVERLAPS #-} Show Env where
    show env = "\\{"++ intercalate ",\\," (map show env) ++"\\}"
instance {-# OVERLAPS #-} Show (Nm,Clos) where
    show (x,cl) = x++"{\\mapsto}"++show cl
instance Show Clos where
    show (Cl e env) = "\\langle "++show e++","++show env++"\\rangle "

instance {-# OVERLAPS #-} IHaskellDisplay Env where
    display e = display [latex $ "$"++show e++"$"]
instance {-# OVERLAPS #-} IHaskellDisplay (Nm,Clos) where
    display e = display [latex $ "$"++show e++"$"]
instance IHaskellDisplay Clos where
    display e = display [latex $ "$"++show e++"$"]

In [8]:
env01 = [ ("f", Cl (Lam "z" $ Var "z") []) ]

In [9]:
env01

In [10]:
env02 = [ ( "x", Cl (Lam "z" $ Var "z")               []   ),
          ( "y", Cl (Lam "x" $ Var "f" `App` Var "x") env01) ]

In [11]:
env02

In [12]:
mapM display it -- 직전 실행한 값계산 환경의 각 대응 항목을 한줄씩 디스플레이

In [13]:
:type lookup

lookup :: forall a b. Eq a => a -> [(a, b)] -> Maybe b

In [14]:
lookup "y" env02 -- Maybe Env 디스플레이 인스턴스 선언 없으므로 텍스트 출력
lookup "z" env02 -- 환경에 없는 변수 참조

Just \langle \lambda x.f\;x,\{f{\mapsto}\langle \lambda z.z,\{\}\rangle \}\rangle

Nothing

In [15]:
lookupEnv :: Nm -> Env -> [Clos]
lookupEnv x env = case lookup x env of Nothing -> []
                                       Just cl -> [cl]

In [16]:
lookupEnv "y" env02 -- 리스트 원소 한줄씩 display되는데 한개이므로

In [17]:
lookupEnv "z" env02 -- 빈 리스트라 아무것도 display되지 않음

In [18]:
bigStepCBV :: Clos -> [Clos]
bigStepCBV (Cl (Var x)     env) = lookupEnv x env
bigStepCBV (Cl (Lam x e)   env) = undefined -- 연습문제로 작성
bigStepCBV (Cl (App e1 e2) env) = undefined -- 연습문제로 작성

In [19]:
bigStepCBV (Cl (Var "y") env02) -- 큰걸음 실행 결과 있음 (환경에 있는 변수)

In [20]:
bigStepCBV (Cl (Var "z") env02) -- 큰걸음 실행 결과 없음 (환경에 없는 변수)

In [21]:
-- bigStepCBV (Cl (Lam ...) ...) -- 연습문제로 작성한 코드 관련 테스트 작성

In [22]:
-- bigStepCBV (Cl (App ...) ...) -- 연습문제로 작성한 코드 관련 테스트 작성

여기까지 인자 먼저 값계산의 큰걸음 의미구조(그림 \ref{fig:bigStepCBV})를 하스켈로 옮긴 함수 `bigStepCBV`를 작성하여 테스트해 보았다면,
또 하나의 연습문제로 적용 먼저 값계산의 큰걸음 의미구조(그림 \ref{fig:bigStepCBN})를 하스켈로 옮긴 함수 `bigStepCBN`도 마찬가지 방법으로
작성하고 테스트해 볼 수 있을 것이다. 같은 클로저를 `bigStepCBN`으로 실행하면 정상적으로 실행되어 결과값이 있는데
`bigStepCBV`는 정상적으로 실행되지 않고 결과값이 없는 경우가 없을지 생각해 보고 있다면 그러한 예시를 구성해 테스트해 보라.
또한 같은 클로저를 `bigStepCBN`과 `bigStepCBV`로 실행했을 때 모두 정상 실행되지만 두 결과값이 똑같지는 않은 경우는 없는지
생각해 보고 있다면 그러한 예시를 구성해 테스트해 보라.

In [23]:
bigStepCBN :: Clos -> [Clos]
bigStepCBN (Cl (Var x)     env) = undefined -- 연습문제로 작성
bigStepCBN (Cl (Lam x e)   env) = undefined -- 연습문제로 작성
bigStepCBN (Cl (App e1 e2) env) = undefined -- 연습문제로 작성

In [24]:
-- 연습문제로 작성한 bigStepCBN에 대한 테스트

# 값계산 환경을 활용한 작은걸음 의미구조
\label{sec:evalCEK}
이 장을 시작하며 \ref{sec:evalSmallStep}절 앞부분에서는 줄임(reduction)과 값계산(evaluation)을 연관지어 이해하기 좋도록
타입없는 람다계산법의 값계산을 맥락적 의미구조로 설명하였다. 그런데, 이런 치환 기반의 의미구조를 그대로 구현한다면 불필요한 치환이
발생하는 단점이 있다. 이런 단점이 해소된 값계산 환경을 활용하는 방식의 큰걸음 의미구조를 \ref{sec:evalBigStep}절에서 살펴보았다.
환경을 활용하는 큰걸음 의미구조에 더 직접적으로 대응되는 작은걸음 의미구조도 생각해 볼 수 있다.
이러한 작은걸음 의미구조가 바로 CEK기계\cite{Felleisen86a}이다.\footnote{여기서는
    조금 단순화된 형태의 다음과 같은 하스켈 코드를 참고해 정리하였다.\\$\phantom{AAx}$
    \url{https://matt.might.net/articles/cek-machines/} }
참고로, CEK는 코드(Code), 환경(Environment), 나중에 할 일(Kontinuation)의 두문자어(acronym)이다.
코드와 환경은 이미 익숙한 람다식과 값계산 환경을 말하므로 \`\`나중에 할 일\'\'(영어: continuation, 독일어: Kontinuation)의 개념만 알면
CEK기계의 작동 원리를 이해할 수 있다.

우선 산술식으로 나중에 할 일의 개념을 알아보자.
산술식 $(\underline{3 + 4}) + (5 + 6)$에서 밑줄 친 부분을 지금 계산하고 있다면,
전체 식을 계산하기 위해 나중에 할 일은 $\bullet + (5 + 6)$처럼 값계산 맥락(evaluation context)과 같은 형태로 표현할 수 있다.
나중에 할 일 $\bullet + (5 + 6)$는 지금 진행하는 계산의 결과를 $\bullet$ 부분에 넣어서 완성되는 전체 식을 계산하라는 뜻이다.
조금 더 큰 산술식 $(1 + 2) + ((\underline{3 + 4}) + (5 + 6))$에서 마찬가지로
밑줄 친 부분을 지금 계산하고 있다면, 전체 식을 계산하기 위해 나중에 할 일을
한꺼번에 $(1 + 2) + (\bullet + (5 + 6))$처럼 표현할 수도 있고,
또 $\bullet + (5 + 6)$와 $(1 + 2) + \bullet$처럼 차례대로 두 개로 나누어 표현할 수도 있을 것이다.
정리하면, 산술식에서 나중에 할 일($\kappa$)이란 일련의 값계산 맥락들로 표현할 수 있다. 그리고 나중에 할 일 목록의
가장 마지막은 항상 전체 식의 값을 나타내는 맥락인 $\bullet$으로 항상 끝마친다고 생각할 수 있으므로,
산술식에 대한 나중에 할 일($\kappa$)의 문법구조를
$\kappa ::= \mathcal{E},\kappa \mid \bullet$로 정의할 수 있다.

환경 기반의 의미구조에서는 람다식($e$)과 값계산 환경($\rho$)을 짝지어 계산을 진행하므로,
CEK기계에서 \`\`나중에 할 일\'\'은 람다식에 대한 맥락($\mathcal{E}$)과 값계산 환경($\rho$)를 짝지은
$\langle\mathcal{E},\rho\rangle$와 같은 형태를 차례로 나열하여 표현한다.
또한, 나중에 할 일의 마지막은 계산하는 닫힌식 전체의 값을 나타내는 $\langle\bullet,\{\}\rangle$이므로,
CEK 기계에서 나중에 할 일($\kappa$)의 문법구조를
$\kappa ::= \langle\mathcal{E},\rho\rangle,\kappa \mid \langle\bullet,\{\}\rangle$로 정의할 수 있다.
CEK기계는 
$\langle e_1,\rho_1\rangle;\kappa_1 \;\longmapsto\; \langle e_2,\rho_2\rangle;\kappa_2 \;\longmapsto\;\ldots$
와 같이 작은걸음으로 계산이 진행되며 $v;\langle\bullet,\{\}\rangle$의 형태일 때 $v$라는 값으로 계산이 정상 종료된 것으로 본다.

이제부터는 인자 먼저(call-by-value) 값계산과 적용 먼저(call-by-name) 값계산을 규정하는 CEK기계의 구체적인 작은걸음 규칙을 살펴보기로 하자.

## 인자 먼저 값계산 CEK기계
인자 먼저 값계산 CEK기계(CEK$_\mathrm{v}$)의 작은걸음 규칙은 아래와 같다.
\vspace*{-1em}
\begingroup
\addtolength{\jot}{-.5ex}
\begin{align*}
\langle x,\rho\rangle;\kappa
        &\longmapsto
\rho(x);\kappa 
& \text{(\textsc{Var})}
\\
\langle e_1~e_2,\rho\rangle;\kappa
        &\longmapsto
\langle e_1,\rho\rangle;\langle \bullet~e_2,\rho\rangle,\kappa
& \text{(\textsc{App}$_1$)}
\\
\langle \lambda x.e,\rho_1\rangle;\langle \bullet~e_2,\rho\rangle,\kappa
        &\longmapsto
\langle e_2,\rho\rangle;\langle(\lambda x.e)\,\bullet,\rho_1\rangle,\kappa
& \text{(\textsc{App}$_2$)}
\\
v_2;\langle(\lambda x.e)\,\bullet,\rho_1\rangle,\kappa
        &\longmapsto
\langle e,\{x{\mapsto}v_2\}\rho_1\rangle;\kappa
& \text{(\textsc{Beta}$_\mathrm{v}$)}
\end{align*}
\vspace*{-2em}
\endgroup

\noindent
위의 네 가지 규칙을 좀더 자세히 설명하면 다음과 같다.

- 첫째, 변수에 대한 규칙(\textsc{Var})으로 한걸음 진행하면, 변수에 대응되는 값을 환경에서 참조한다.
- 둘째, 함수적용식의 함수 부분에 대한 규칙(\textsc{App}$_1$)으로 한걸음 진행하면,
  우선 왼쪽의 함수 부분($e_1$)부터 현재 값계산 환경($\rho$)에서 지금부터 계산하고
  나중에 할 일 목록에 앞으로 오른쪽의 인자 부분($e_2$)의 계산을 진행하기 위한 항목($\langle\bullet\;e_2,\rho\rangle$)을 추가해 놓는다.
- 셋째, 함수적용식의 인자 부분에 대한 규칙(\textsc{App}$_2$)으로 함수 부분의 계산이 끝난 값의 형태($\langle\lambda x.e,\rho_1$)일 때
  나중의 할 일 목록에서 가장 앞에 위치한 $\langle\bullet\;e_2,\rho\rangle$를 제거하며 한걸음 진행하면,
  지금부터 함수 적용식의 인자 부분($e_2$)의 계산을 진행하되
  나중의 할 일 목록에 앞으로 함수 호출을 처리하기 위한 항목($\langle(\lambda x.e)\,\bullet,\rho_1\rangle$)을 추가해 놓는다.
- 넷째, 함수 호출을 처리하는 규칙(\textsc{Beta}$_\mathrm{v}$)으로 인자 부분의 계산이 끝난 값($v_2$)의 형태일 때 
  나중의 할 일 목록에서 가장 앞에 위치한 $\langle(\lambda x.e)\,\bullet,\rho_1\rangle$를 제거하며 한걸음 진행하면,
  함수 부분의 값을 나타내는 클로저에 포함된 기존의 환경($\rho_1$)에
  추가로 함수 파라메터($x$)에 인자 부분이 계산된 값($v_2$)을 대응시켜 확장한 새로운 환경에서
  지금부터 함수 몸체($e$)의 계산을 진행한다.

\noindent
참고로, 마지막의 넷째 규칙(\textsc{Beta}$_\mathrm{v}$)에 나타나는 $v_2$는 환경 기반의 인자 먼저 값계산에서 닫힌식의 계산이 완료된 형태의 클로저,
즉 $\langle\lambda x_2.e_2,\rho_2\rangle$와 같은 형태이다.

## 적용 먼저 값계산 CEK기계
적용 먼저 값계산 CEK기계(CEK$_\mathrm{n}$)의 작은걸음 규칙은 아래와 같다.
\vspace*{-1em}
\begingroup
\addtolength{\jot}{-.5ex}
\begin{align*}
\langle x,\rho\rangle;\kappa
        &\longmapsto
\rho(x);\kappa
& \text{(\textsc{Var})}
\\
\langle e_1~e_2,\rho\rangle;\kappa
        &\longmapsto
\langle e_1,\rho\rangle;\langle\bullet~e_2,\rho\rangle,\kappa
& \text{(\textsc{App}$_1$)}
\\
\langle \lambda x.e,\rho_1\rangle;\langle\bullet~e_2,\rho\rangle,\kappa
        &\longmapsto
\langle e,\{x{\mapsto}\langle e_2,\rho\rangle\}\rho_1\rangle;\kappa
& \text{(\textsc{Beta}$_\mathrm{n}$)}
\end{align*}
\vspace*{-2em}
\endgroup

\noindent
CEK$_\mathrm{n}$의 \textsc{Var}과 \textsc{App}$_1$규칙은 앞서 살펴본 CEK$_\mathrm{v}$와 마찬가지지만
함수 적용식의 인자 부분에 대한 계산과 함수 호출에 대한 규칙에서 차이가 있다.
CEK$_\mathrm{v}$에서는 함수적용식의 인자 부분($e_2$)에 대한 계산을 \textsc{App}$_2$규칙으로 먼저 진행하고 나서
\textsc{Beta}$_v$규칙에 따라 함수를 호출하는 반면, CEK$_\mathrm{n}$에서는 함수적용식의 함수 부분($e_1$)에 대한 계산이 끝나면
곧바로 \textsc{Beta}$_n$규칙으로 인자 부분($e_2$)의 계산을 진행하지 않고 그대로 클로저를 구성해 함수를 호출한다.

## CEK기계의 작은걸음 함수를 하스켈로 작성하기
CEK기계의 작은걸음 규칙을 하스켈로 옮기기 위해
나중에 할 일(`Kont`)과 이를 구성하는 일부분인 맥락(`Ctx`)을 나타내는
타입을 다음과 같이 선언한다.

In [25]:
type Kont = [(Ctx,Env)] -- 나중에 할 일을 Ctx와 Env의 순서쌍 리스트로 표현
data Ctx = Done | App1 Expr | App2 Expr  deriving (Eq,Ord)

\noindent
위에 선언된 데이터 타입 `Ctx`는 일반적인 값계산 맥락보다 단순하다.
일반적으로 값계산 맥락은 $e_1\,(\bullet\;e_2)$와 같이 구멍을 나타내는 $\bullet$이 두 겹 이상 안쪽으로 파고든 부분식의 위치에 나타날 수 있으므로 재귀적인 데이터 타입으로 선언해야 한다.
하지만 CEK기계의 계산 과정에서는 이렇게 여러 겹 안쪽에 구멍이 뚫린 하나의 복잡한 값계산 맥락 대신에 이를 두 개의 단순한 맥락 $\bullet\;e_2$와 $e_1\;\bullet$로 나누어 순차적으로 적용하는 방식으로 계산이 진행된다.
따라서 전체($\bullet$) 또는 함수적용식의 왼쪽($\bullet\;e$)이나 오른쪽($e\;\bullet)$에 구멍이 나타나는 세 가지 단순한 맥락만을 고려하면 되므로 `Ctx`를 재귀적으로 정의할 필요가 없다.

In [26]:
instance Show Ctx where -- LaTeX 소스코드 생성하는 Show 인스턴스 직접 선언
  showsPrec _ Done = showString "\\bullet "
  showsPrec p (App1 e2) = showParen (p > 9) $
      showString "\\bullet \\;" . showsPrec 10 e2
  showsPrec p (App2 e1) = showParen (p > 9) $
      showsPrec 9 e1 . showString "\\; \\bullet"

import IHaskell.Display (latex)
instance IHaskellDisplay Ctx where -- 노트북에 디스플레이하는 인스턴스 선언
    display ctx = display [latex $ "$"++show ctx++"$"]

In [27]:
App1 (Var "x" `App` Var "y")
App2 (Lam "x" $ Var "x")

In [28]:
import Data.List (intercalate)
instance {-# OVERLAPS #-} Show Kont where
    show k = intercalate ",\\," (map show k)
instance {-# OVERLAPS #-} Show (Clos,Kont) where
    show (cl,k) = show cl++";\\,"++show k
instance {-# OVERLAPS #-} Show (Ctx,Env) where
    show (ctx,env) = "\\langle "++show ctx++","++show env++"\\rangle "
instance {-# OVERLAPS #-} IHaskellDisplay Kont where
    display k = display [latex $ "$"++show k++"$"]
instance {-# OVERLAPS #-} IHaskellDisplay (Clos,Kont) where
    display e = display [latex $ "$"++show e++"$"]
instance {-# OVERLAPS #-} IHaskellDisplay (Ctx,Env) where
    display e = display [latex $ "$"++show e++"$"]

In [29]:
(Done,[]) :: (Ctx,Env) -- Kont의 마지막에서 전체 식의 계산을 끝냄을 나타낸다

In [30]:
k03 = [ (App1 (Var "f"),        env01),
        (App2 (Lam "x" $ Var "x"), []),
        (Done,                     []) ]
k03 :: Kont

In [31]:
e03 = Lam "y" (Var "y" `App` Var "f")
(Cl e03 env01, k03) :: (Clos,Kont) -- 클로저와 나머지 할 일의 순서쌍

$~$\vspace*{-2.5em}\newline

지금까지 CEK기계를 하스켈로 옮기는 데 필요한 타입을 선언하고
노트북 환경에서 CEK기계의 작은걸음 규칙에 나타난 것과 마찬가지로 출력되도록 클래스 인스턴스를 선언하였다.
이제 인자 먼저 값계산을 따르는 CEK$_\mathrm{v}$와 적용 먼저 값계산을 따르는 CEK$_\mathrm{n}$의
작은걸음 함수 `stepCEKv`와 `stepCEKn`을 작성해 보라.

In [32]:
stepCEKv :: (Clos,Kont) -> [(Clos,Kont)] -- CEKv 작은걸음 함수
stepCEKv _ = undefined -- 여러 개의 등식으로 나누어 작성해도 좋다

In [33]:
stepCEKn :: (Clos,Kont) -> [(Clos,Kont)] -- CEKn 작은걸음 함수
stepCEKn _ = undefined -- 여러 개의 등식으로 나누어 작성해도 좋다

In [34]:
stepCEKv :: (Clos,Kont) -> [(Clos,Kont)]
stepCEKv (Cl (Var x)     env, k) = [(v,k) | v <- lookupEnv x env]
stepCEKv (Cl (App e1 e2) env, k) = [(Cl e1 env, (App1 e2,env):k)]
stepCEKv (Cl (Lam x e)  env1, (App1 e2,env):k) = [(Cl e2 env, (App2 (Lam x e), env1):k)]
stepCEKv (v2@(Cl (Lam x2 e2) env), (App2 (Lam x e),env1):k) = [(Cl e ((x,v2):env1), k)] 
stepCEKv _                       = []

stepCEKn :: (Clos,Kont) -> [(Clos,Kont)]
stepCEKn (Cl (Var x)     env, k) = [(v,k) | v <- lookupEnv x env]
stepCEKn (Cl (App e1 e2) env, k) = [(Cl e1 env, (App1 e2,env):k)]
stepCEKn (Cl (Lam x e)  env1, (App1 e2,env):k) = [(Cl e ((x,Cl e2 env):env1), k)]
stepCEKn _                       = []

In [35]:
hat f es = concat [f e | e <-es]
stepCEKv' = hat stepCEKv
stepCEKn' = hat stepCEKn
:type stepCEKv'
:type stepCEKn'

stepCEKv' :: [(Clos, Kont)] -> [(Clos, Kont)]

stepCEKn' :: [(Clos, Kont)] -> [(Clos, Kont)]

In [36]:
e0 = Lam "z" (Var "z") `App` Lam "y" (Var "y")
env0 = [] :: Env
k0 = [(Done,env0)]
(Cl e0 env0, k0)

\noindent
작은걸음 함수 `stepCEKv`와 `stepCEKn`을 제대로 작성했다면
각각의 확장된 함수 `stepCEKv'`와 `stepCEKn'`를 `e0`에 반복적으로 적용하여
아래와 같이 서로 다른 두 값계산 전략에 따른 계산의 진행 과정을 확인해 볼 수 있을 것이다.

In [37]:
takeWhile  (not . null) $ iterate stepCEKv' [(Cl e0 env0, k0)]
length it - 1  -- 작은걸음 횟수

4

In [38]:
takeWhile  (not . null) $ iterate stepCEKn' [(Cl e0 env0, k0)]
length it - 1  -- 작은걸음 횟수

3

치환 기반의 의미구조에서는 $(\lambda z.z)(\lambda y.y)\longmapsto (\lambda y.y)$로
인자 먼저든 적용 먼저든 한 작은걸음으로 값계산 과정이 똑같다.
그런데 CEK$_\textrm{v}$에서 `e0`에 대한 작은걸음 횟수가 CEK$_\textrm{n}$보다 1회 더 많은 이유는
CEK$_\textrm{v}$의 경우 \textsc{Beta}$_v$전에 \textsc{App}$_2$를 거쳐야 하기 때문이다.

연습문제로, 조금 더 복잡한 다음의 식 `e01`에 마찬가지로 확장된 함수 `stepCEKv'`와 `stepCEKn'`를
반복적으로 적용해 보고 서로 다른 두 값계산 전략에 따른 계산의 진행 과정이 어떻게
다른지 확인해 보고 그런 차이가 나타나는 이유를 분석해 보라.

In [39]:
e01 = Lam "x" (foldl1 App $ replicate 3 (Var "x")) `App` e0
(Cl e01 env0, k0)

In [40]:
-- takeWhile  (not . null) $ iterate stepCEKv' [(Cl e01 env0, k0)]
-- length it - 1  -- 작은걸음 횟수

In [41]:
-- takeWhile  (not . null) $ iterate stepCEKn' [(Cl e01 env0, k0)]
-- length it - 1  -- 작은걸음 횟수 

## CESK기계와 필요한 만큼 값계산(call-by-need evaulation) 
지금까지 살펴본 바와 같이 값계산 환경을 활용하면 계산 과정에서 불필요한 부분의
변수 이름까지 치환하는 낭비가 없어진다. 하지만, 환경 기반의 의미구조라고 해서
필요한 부분에 같은 계산이 여러 번 중복되는 낭비까지 자동으로 없어지지는 않는다.
특히 인자 계산보다 함수 적용을 우선하는 적용 먼저(call-by-name) 값계산에서 중복된 계산이 발생할 여지가 많다.
간단한 예시를 통해 어떤 경우에 중복된 계산이 발생할 수 있는지 확인하고
그런 중복된 계산을 줄일 수 있는 방법에 대해서도 생각해 보자.

이 장에서는 순수한 람다식에 대한 의미구조를 다루고 있지만 설명의 편의를 위해 덧셈식도 있다고 가정하고
다음과 같은 클로저에 대한 계산을 살펴보자.\vspace*{-1.25em}
\begingroup
\addtolength{\jot}{-.5ex}
\begin{align*}
\langle x_1+x_1,\{x_1{\mapsto}\langle 1+1+1,\rho\rangle\}\rangle;\,\kappa
\longmapsto \cdots \longmapsto\;& \langle 1+1+1,\;\ldots\rangle;\,\langle \bullet+x_1,\,\ldots\rangle,\kappa \\
\longmapsto~\cdots~\cdots~\longmapsto\;& \langle 3,\;\ldots\rangle;\,\langle \bullet+x_1,\,\ldots\rangle,\kappa \\
\longmapsto \cdots \longmapsto \langle x_1,\;\ldots\rangle;\,\langle 3+\bullet,\,\ldots\rangle,\kappa
\longmapsto\;& \langle 1+1+1,\;\ldots\rangle;\,\langle 3+\bullet,\,\ldots\rangle,\kappa \\
\longmapsto~\cdots~\cdots~\longmapsto\;& \langle 3+3,\;\ldots\rangle;\,\kappa
\;\longmapsto\; \langle 6,\;\ldots\rangle;\,\kappa
\\[-2.5em]
\end{align*}
\endgroup
위 계산 과정에서는 산술식 $1+1+1$을 정수값 $3$으로 두 번 중복해서 계산한다.
이런 중복된 계산을 피하기 위한 한 가지 개선 방법은 환경에서 $x_1$을 참조하여
계산한 결과를 환경에 반영하여 이후로는 $x_1$을
$\langle 1+1+1,\rho\rangle$대신 $\langle 3,\rho\rangle$에 대응시킨
환경을 활용하도록 대략 아래와 같은 방식으로 계산을 진행하는 것이다.\vspace*{-1.25em}
\begingroup
\addtolength{\jot}{-.5ex}
\begin{align*}
\langle x_1+x_1,\{x_1{\mapsto}\langle 1+1+1,\rho\rangle\}\rangle;\,\kappa
\longmapsto \cdots \longmapsto\;& \langle 1+1+1,\;\ldots\rangle;\,\langle \bullet+x_1,\,\ldots\rangle,\kappa \\
\longmapsto~\cdots~\cdots~\longmapsto\;& \langle 3,\{x_1{\mapsto}\langle 3,\rho\rangle\}\rangle;\,\langle \bullet+x_1,\,\ldots\rangle,\kappa \\
\longmapsto \cdots \longmapsto \langle x_1,\;\ldots\rangle;\,\langle 3+\bullet,\,\ldots\rangle,\kappa
\longmapsto\;& \langle 3,\;\ldots\rangle;\,\langle 3+\bullet,\,\ldots\rangle,\kappa \\
\longmapsto\;& \langle 3+3,\;\ldots\rangle;\,\kappa
\;\longmapsto\; \langle 6,\;\ldots\rangle;\,\kappa
\\[-2.5em]
\end{align*}
\endgroup

지금까지 같은 변수 이름을 여러 번 참조하는 경우의 중복된 계산을 줄이는 방법에 대해 알아보았다.
그런데 이런 방법만으로는 아래와 같이 서로 다른 이름으로 같은 내용을 참조하는 경우에 중복되는 계산을 피하지 못한다.
왜냐하면 $x_1$을 참조한 $1+1+1$를 $3$으로 계산한 결과를 환경에 반영한다 하더라도
여전히 $x_2$는 $3$이 아닌 $1+1+1$을 참조하여 대략 아래와 같은 과정으로 값계산이 전개될 것이기 때문이다.\vspace*{-1.5ex}
\begingroup
\begin{align*}
& \langle x_1+x_2,\{x_1{\mapsto}\langle 1+1+1,\rho\rangle,x_2{\mapsto}\langle 1+1+1,\rho\rangle\}\rangle;\,\kappa
\\ \longmapsto~ &
\langle x_1,\{x_1{\mapsto}\langle 1+1+1,\rho\rangle,x_2{\mapsto}\langle 1+1+1,\rho\rangle\}\rangle;\,\langle\bullet+x_2,\ldots\rangle,\kappa
\\ \longmapsto~ &
\langle 1+1+1,\{x_1{\mapsto}\langle 1+1+1,\rho\rangle,x_2{\mapsto}\langle 1+1+1,\rho\rangle\}\rangle;\,\langle\bullet+x_2,\ldots\rangle,\kappa
\\ \longmapsto\cdots\longmapsto~&
\langle 3+x_2,\{x_1{\mapsto}\langle 3,\rho\rangle,x_2{\mapsto}\langle 1+1+1,\rho\rangle\}\rangle;\,\kappa
\\ \longmapsto~&
\langle x_2,\{x_1{\mapsto}\langle 3,\rho\rangle,x_2{\mapsto}\langle 1+1+1,\rho\rangle\}\rangle;\,\langle 3+\bullet,\,\ldots\rangle,\kappa
\\ \longmapsto~&
\langle 1+1+1,\{x_1{\mapsto}\langle 3,\rho\rangle,x_2{\mapsto}\langle 1+1+1,\rho\rangle\}\rangle;\,\langle 3+\bullet,\,\ldots\rangle,\kappa
\\ \longmapsto\cdots\longmapsto~&
\langle 3+3,\{x_1{\mapsto}\langle 3,\rho\rangle,x_2{\mapsto}\langle 3,\rho\rangle\}\rangle;\,\kappa
~\longmapsto~
\langle 6,\;\ldots\rangle;\,\kappa
\\[-1.5em]
\end{align*}
\endgroup

CEK기계에 주소로 접근할 수 있는 \underline{저장공간(Store)}이 추가된 CESK기계\cite{Felleisen87CESK}는
서로 다른 이름의 변수가 같은 내용을 공유한다는 개념이 표현 가능하다. 따라서,
$x_1$과 $x_2$에 대응되는 클로저를 저장공간에서 같은 주소($\ell_1$)를 통해 공유함으로써
방금 살펴본 값계산 과정에서 중복된 계산을 아래와 같이 피할 수 있다.\vspace*{-1.5ex}
\begingroup
\begin{align*}
& \langle x_1+x_2,\{x_1{\mapsto}\ell_1,x_2{\mapsto}\ell_1\}\rangle;\,
\underline{\{\ell_1{\mapsto}\langle 1+1+1,\rho\rangle\}};\,\kappa
\\ \longmapsto~ &
\langle x_1,\{x_1{\mapsto}\ell_1,x_2{\mapsto}\ell_1\}\rangle;\,
\underline{\{\ell_1{\mapsto}\langle 1+1+1,\rho\rangle\}};\,\langle\bullet+x_2,\ldots\rangle,\kappa
\\ \longmapsto~ &
\langle 1+1+1,\{x_1{\mapsto}\ell_1,x_2{\mapsto}\ell_1\}\rangle;\,
\underline{\{\ell_1{\mapsto}\langle 1+1+1,\rho\rangle\}};\,\langle\bullet+x_2,\ldots\rangle,\kappa
\\ \longmapsto\cdots\longmapsto~&
\langle 3+x_2,\{x_1{\mapsto}\ell_1,x_2{\mapsto}\ell_1\}\rangle;\,
\underline{\{\ell_1{\mapsto}\langle 3,\rho\rangle\}};\,\kappa
\\ \longmapsto~&
\langle x_2,\{x_1{\mapsto}\ell_1,x_2{\mapsto}\ell_1\}\rangle;\,
\underline{\{\ell_1{\mapsto}\langle 3,\rho\rangle\}};\,\langle 3+\bullet,\,\ldots\rangle,\kappa
\\ \longmapsto~&
\langle 3,\{x_1{\mapsto}\ell_1,x_2{\mapsto}\ell_1\}\rangle;\,
\underline{\{\ell_1{\mapsto}\langle 3,\rho\rangle\}};\,\langle 3+\bullet,\,\ldots\rangle,\kappa
\\ \longmapsto~&
\langle 3+3,\{x_1{\mapsto}\ell_1,x_2{\mapsto}\ell_1\}\rangle;\,
\underline{\{\ell_1{\mapsto}\langle 3,\rho\rangle\}};\,\kappa
\,\longmapsto\,
\langle 6,\;\ldots\rangle;\,\underline{\cdots};\,\kappa
\\[-1.5em]
\end{align*}
\endgroup

CEK와 CESK기계의 차이점을 간략히 정리하면 그림 \ref{fig:CEKvsCESK}과 같다.
CEK기계에서는 환경($\rho\in\texttt{Env}$)으로부터 변수의 이름($x$)에 대응되는 클로저를
$\rho(x)$와 같이 참조한다. 한편, CESK기계에서는 환경($\rho\in\texttt{Env\textquotesingle}$)으로부터
변수의 이름($x$)에 대응되는 클로저의 주소를 $\rho(x)$로 참조하여
저장공간($\sigma\in\texttt{Store}$)으로부터 해당 주소의 클로저를 $\sigma(\rho(x))$와 같이 참조한다.

\begin{align*}
\text{CEK기계}~&\text{\fbox{$\langle e,\rho\rangle;\,\kappa \longmapsto \langle e',\rho'\rangle;\,\kappa'$}} &
\text{CESK기계}&~\text{\fbox{$\langle e,\rho\rangle;\,\sigma;\,\kappa \longmapsto \langle e',\rho'\rangle;\,\sigma';\,\kappa'$}}
\\
\cdot\longmapsto\cdot \,\subset\,&\;
((\texttt{Expr}\!\times\!\texttt{Env})\times\texttt{Kont})^2 &
\cdot\longmapsto\cdot \,\subset\,&\;
((\texttt{Expr}\!\times\!\texttt{Env\textquotesingle})\times\texttt{Store}\times\texttt{Kont\textquotesingle})^2
\\
\rho\in~\texttt{Env} \;&=\; \texttt{Nm} \xrightharpoonup{\,_\textrm{fin}\,} \texttt{Expr}\times\texttt{Env} &
\rho\in~\texttt{Env\textquotesingle} &\,=\; \texttt{Nm} \xrightharpoonup{\,_\textrm{fin}\,} \texttt{Addr}
\\ & &
\sigma\in\texttt{Store} &\,=\; \texttt{Addr} \xrightharpoonup{\,_\textrm{fin}\,} \texttt{Expr}\times\texttt{Env\textquotesingle}
\end{align*}

$~$

CEK$_\textrm{v}$에 대응되는 인자 먼저 값계산 CESK기계(CESK$_\mathrm{v}$)의 작은걸음 규칙은
\vspace*{-1em}
\begingroup
\addtolength{\jot}{-.2ex}
\begin{align*}
\langle x,\rho\rangle;\sigma;\kappa
        &\longmapsto
\sigma(\rho(x));\sigma;\kappa 
& \text{(\textsc{Var})}
\\
\langle e_1~e_2,\rho\rangle;\sigma;\kappa
        &\longmapsto
\langle e_1,\rho\rangle;\sigma;\langle \bullet~e_2,\rho\rangle,\kappa
& \text{(\textsc{App}$_1$)}
\\
\langle \lambda x.e,\rho_1\rangle;\sigma;\langle \bullet~e_2,\rho\rangle,\kappa
        &\longmapsto
\langle e_2,\rho\rangle;\sigma;\langle(\lambda x.e)\,\bullet,\rho_1\rangle,\kappa
& \text{(\textsc{App}$_2$)}
\\
v_2;\sigma;\langle(\lambda x.e)\,\bullet,\rho_1\rangle,\kappa
        &\longmapsto
\langle e,\{x{\mapsto}\ell\}\rho_1\rangle;\{\ell{\mapsto}v_2\}\sigma;\kappa
& {\scriptstyle\left(\,\ell\,\notin\,\mathrm{dom}(\sigma)\,\right)}\,\text{(\textsc{Beta}$_\mathrm{v}$)}
\\[-2.5em]
\end{align*}
\endgroup
위와 같으며, CEK$_\textrm{n}$에 대응되는 적용 먼저 값계산 CESK기계(CESK$_\mathrm{n}$)의 규칙은 아래와 같다.
\vspace*{-2.5em}
\begingroup
\addtolength{\jot}{-.2ex}
\begin{align*}
\langle x,\rho\rangle;\sigma;\kappa
        &\longmapsto
\sigma(\rho(x));\sigma;\kappa 
& \text{(\textsc{Var})}
\\
\langle e_1~e_2,\rho\rangle;\sigma;\kappa
        &\longmapsto
\langle e_1,\rho\rangle;\sigma;\langle \bullet~e_2,\rho\rangle,\kappa
& \text{(\textsc{App}$_1$)}
\\
\langle \lambda x.e,\rho_1\rangle;\sigma;\langle\bullet~e_2,\rho\rangle,\kappa
        &\longmapsto
\langle e,\{x{\mapsto}\ell\}\rho_1\rangle;\{\ell{\mapsto}\langle e_2,\rho\rangle\}\sigma;\kappa
& \text{(\textsc{Beta}$_\mathrm{n}$)} \\[-1.5ex]
&
& \!\!\!\!\!
{\scriptstyle\left(\,\ell\,\notin\,\mathrm{dom}(\sigma)\,\right)}
\!\!\!\!\phantom{\text{(\textsc{Beta}$_\mathrm{n}$)}}
\\[-3em]
\end{align*}
\endgroup

변수 이름에 계산이 끝나지 않은 클로저가 대응될 수도 있는 CESK$_\mathrm{n}$에서는
같은 이름으로 여러 번 혹은 다른 이름들로 똑같은 주소에 있는 클로저를
참조하는 경우에 중복된 계산을 하게 된다. 이렇게 중복된 계산을 유발하는
\textsc{Var}규칙을 다음의 두 규칙 \textsc{Var}$_1$과 \textsc{Update}로 대체하면,
\textsc{Var}$_1$규칙으로 참조한 계산 결과를 \textsc{Update}규칙으로 공유하여
그 이후의 계산 과정에서 같은 주소를 참조하는 이름에 대한 중복된 계산을 피할 수 있다.
\vspace*{-1em}
\begingroup
\addtolength{\jot}{-.2ex}
\begin{align*}
\langle x,\rho\rangle;\,\sigma;\,\kappa
        &\longmapsto
\sigma(\rho(x));\,\sigma;\,\rho(x){\mapsto}\bullet,\kappa 
& \text{(\textsc{Var}$_1$)}
\\
v;\,\sigma;\,\ell{\mapsto}\bullet,\kappa
        &\longmapsto
v;\,\{\ell{\mapsto}\bullet\}\sigma;\,\kappa
& \text{(\textsc{Update})}
\\[-2.5em]
\end{align*}
\endgroup
여기서 주목할 점은 기존의 값계산 맥락을 포함하는 나중에 할 일 구성 요소 이외에,
지금 집중하고 있는 계산이 끝난 결과값을 저장공간의 특정 주소에 반영하라는 의미의,
$\ell{\mapsto}\bullet$ 형태의 새로운 구성 요소가
위의 두 규칙 \textsc{Var}$_1$과 \textsc{Update}의 나중에 할 일 목록에 나타난다는 것이다.

정리하면, CESK$_\mathrm{n}$에서 \textsc{Var}규칙을 제외하고 위의 두 규칙으로 대체한
총 네 개의 작은걸음 규칙(\textsc{Var}$_1$, \textsc{Update}, \textsc{App}$_1$, \textsc{Beta}$_\mathrm{n}$)으로
규정되는 것이 바로 필요한 만큼 값계산 CESK기계(CESK$_z$)이다. 필요한 만큼 값계산(call-by-need evaluation)을
게으른(혹은 느긋한) 값계산(lazy evaluation) 이라고도 부르며 이는 인자 먼저 값계산(call-by-value evaluation)과
동의어인 적극적 값계산(eager evaluation)과 상대되는 용어이다. 그러니까 인자 먼저(call-by-value) 및 적용 먼저(call-by-name) 값계산과 함께 언급할 때는 필요한 만큼(call-by-need) 값계산이라고 부르며, 적극적(eager) 값계산과 함께 언급할 때는 게으른(lazy) 값계산이라고 부른다.

TODO CESK$_z$ 하스켈로 옮겨보면 ... TODO

In [42]:
type Addr = Int
type Env' = [(Nm,Addr)]
data Clos' = Cl' Expr Env' deriving (Eq,Ord)

In [43]:
import qualified Data.HashMap.Strict as M -- M.funcName으로 활용
import Data.HashMap.Strict (HashMap, (!), (!?)) -- 앞에 M. 없이
type Store = HashMap Int Clos'

In [44]:
import Data.List (intercalate)
instance {-# OVERLAPS #-} Show Env' where
  show env = "\\{"++ intercalate ",\\," (map show env) ++"\\}"
instance {-# OVERLAPS #-} Show (Nm,Addr) where
  show (x,l) = x++"{\\mapsto}"++show l
instance Show Clos' where
  show (Cl' e env) = "\\langle "++show e++","++show env++"\\rangle "
instance {-# OVERLAPS #-} Show (Addr,Clos') where
  show (l,cl) = show l++"{\\mapsto}"++show cl
instance {-# OVERLAPS #-} Show Store where
  show s = "\\{"++ intercalate ",\\," (map show $ M.toList s) ++"\\}"

instance {-# OVERLAPS #-} IHaskellDisplay Env where
  display e = display [latex $ "$"++show e++"$"]
instance {-# OVERLAPS #-} IHaskellDisplay (Nm,Addr) where
  display e = display [latex $ "$"++show e++"$"]
instance IHaskellDisplay Clos' where
  display e = display [latex $ "$"++show e++"$"]
instance {-# OVERLAPS #-} IHaskellDisplay (Addr,Clos') where
  display e = display [latex $ "$"++show e++"$"]
instance {-# OVERLAPS #-} IHaskellDisplay Store where
  display e = display [latex $ "$"++show e++"$"]

In [45]:
sigma00 = M.empty :: Store
sigma01 = M.insert 1 (Cl' (Lam "x" $ Var "x") []) sigma00 :: Store
sigma02 = M.insert 2 (Cl' (Lam "z" $ Var "z") []) sigma01 :: Store
sigma03 = M.insert 1 (Cl' (Lam "y" $ Var "y") []) sigma02 :: Store

sigma00
sigma01
sigma02
sigma03

In [46]:
sigma02 ! 2 -- 키에 대응되는 내용을 찾아줌
sigma01 ! 2 -- 키가 없는 경우는 에러 발생

: 

In [47]:
sigma02 !? 2 -- Maybe Clos' 디스플레이 인스턴스 선언되지 않아
sigma01 !? 2 -- 그냥 텍스트 형태로 출력됨

Just \langle \lambda z.z,\{\}\rangle

Nothing

In [48]:
type Kont' = [Either Addr (Ctx,Env')]

In [49]:
import Data.List (intercalate)
instance {-# OVERLAPS #-} Show Kont' where
    show k = intercalate ",\\," (map showEither k)
           where showEither (Left l)  = show l++"{\\mapsto}\\bullet "
                 showEither (Right r) = show r
instance {-# OVERLAPS #-} Show (Clos',Store,Kont') where
    show (cl,s,k) = show cl++";\\,"++show s++";\\,"++show k
instance {-# OVERLAPS #-} Show (Ctx,Env') where
    show (ctx,env) = "\\langle "++show ctx++","++show env++"\\rangle "
instance {-# OVERLAPS #-} IHaskellDisplay Kont' where
    display k = display [latex $ "$"++show k++"$"]
instance {-# OVERLAPS #-} IHaskellDisplay (Clos',Store,Kont') where
    display e = display [latex $ "$"++show e++"$"]
instance {-# OVERLAPS #-} IHaskellDisplay (Ctx,Env') where
    display e = display [latex $ "$"++show e++"$"]

In [50]:
[ Left 1, Right (Done,[]) ] :: Kont'

In [51]:
( Cl' (Var "y") [("y",1)], sigma02, [Left 1, Right (Done,[])] )
  :: (Clos',Store,Kont')

In [52]:
stepCESKz :: (Clos',Store,Kont') -> [(Clos',Store,Kont')]
stepCESKz _ = undefined

\section*{요점정리}
* 타입없는 람다계산법의 비결정적 줄임(reduction)을 나타내는 맥락적 의미구조에
  나타나는 값계산 맥락 및 작은걸음 규칙에 제약을 가하여 인자 먼저(call-by-value) 및
  적용 먼저(call-by-name) 값계산(evaluation)의 의미구조를 규정할 수 있다.
* 람다계산법에 대한 치환 기반의 의미구조는 계산 결과에 활용되지 않는 불필요한 위치의 변수까지도
  모두 치환해야 하므로 프로그래밍언어의 효율적인 구현에는 적합하지 않다.
* 함수 몸체에 나타나는 변수를 실제로 치환하는 대신에 치환 기반의 의미구조에서 계산이 진행되며
  치환되었어야 할 목록을 차곡차곡 모아 놓은 내용을 \'값계산 환경\'(evaluation environment)으로
  구성하여 계산이 진행되며 필요한 위치의 변수만을 값계산 환경에서 참조하는 방식의 의미구조를 정의할 수 있다.
* 값계산 환경 기반의 큰걸음 의미구조를 프로그램으로 옮겨 작성한 소프트웨어를 프로그래밍언어의 인터프리터로 볼 수 있다.
* 값계산 환경 기반의 큰걸음 의미구조에 직접적으로 대응되는 작은걸음 의미구조에 해당하는 것이 바로
  CEK기계다.
* CEK기계의 구성 요소는 코드(Code, 즉 람다식), 환경(Envionment), 나중에 할 일(독일어: Kontinuation, 영어: continuation)이다.
  코드와 환경의 순서쌍이 곧 클로저이므로 CEK기계는 클로저와 나중에 할 일로 구성된다고도 할 수 있다.
* CEK기계에서 \`나중에 할 일\'은 값계산 맥락(evaluation context)과 환경(environment)의 순서쌍들이 차례로 나열된 구조이다.
  이렇게 나중에 할 일을 목록으로 만들어 남겨두고 전체 식의 계산에서 지금 집중해서 계산할 부분식의 계산을 한걸음씩 작은걸음으로 진행하는 방식으로 CEK기계가 동작한다.
* 필요한 만큼만 값계산(call-by-need evaluation)은 적용 먼저 값계산(call-by-name evaluation)과
  기본적으로 비슷하지만 여러 곳에서 활용되는 같은 변수 이름 혹은 같은 서로 다른 변수의 동일한 계산 결과를 공유함으로써
  계산의 중복을 줄이는 장점이 있다.
* 인자 먼저 값계산(call-by-value evaluation)을 적극적 값계산(eager evaluation)
  또는 엄격한 값계산(strict evaluation)으로도 부르며, 필요한 만큼만 값계산(call-by-need evaluation)은
  게으른(혹은  느긋한) 값계산(lazy evaluation)으로도 부른다.
* CEK기계에 주소를 통해 접근할 수 있는 저장공간(Store)이 추가된 CESK기계로
  계산 결과를 공유하는 게으른 값계산의 작은걸음 의미구조를 규정할 수 있다. 

\section*{연습문제}
1. `[18]`-`[24]`셀 중에서 완성되지 않은 큰걸음 함수 `bigStepCBV`와 `bigStepCBN`를 작성하여 테스트해 보라.
1. `[32]`와 `[33]`셀의 `stepCEKv`와 `stepCEKn`을 본문의 작은걸음 규칙대로 작성하여 `[36]`-`[39]`셀의 `e0`에 대해 기대한 대로 동작하는지 확인해 보라.
1. `[39]`셀의 `e01`에 대해 본문에 안내된 대로 `[40]`과 `[41]`셀의 주석을 풀어 실행하여 CEK$_\textrm{v}$와 CEK$_\textrm{n}$의 계산 과정의 차이를 확인해 보고 그 이유를 분석해 보라.

\section*{탐구과제}
1. CEK$_\textrm{v}$와 CEK$_\textrm{n}$처럼 람다계산법을 작은걸음으로 값계산하는 기계인데,
   그보다 더 저수준 구현(컴파일러 작성을 위한 가상머신 등)에 적합한 SECD기계\cite{Landin64SECD}와
   Krivine기계\cite{Krivine07}에 대해 조사해 보라. 참고로 SECD와 Krivine기계가 앞서 고안되었으며
   이후에 이를 단순화하여 CEK$_\textrm{v}$나 CEK$_\textrm{n}$과 같은 형태로 정리된 것으로 이해할 수 있다.
1. CESK$_z$에 대응되는 필요한 만큼 (혹은 게으른) 값계산의 큰걸음 의미구조 $\langle e,\rho\rangle;\sigma\Longmapsto v;\sigma$를 정의해 보고, 이를 바탕으로 하스켈로 람다계산법의 대한 필요한 만큼 (혹은 게으른) 값계산 인터프리터를 작성해 보라.
1. \`\`오프사이드 규칙\'\'(off-side rule)과 \`\`문법 설탕\'\'(syntactic sugar)은 피터 란딘이 정착시킨 프로그래밍언어와 관련한 개념에 대한 전문용어다. 이들에 대해 알아보라. 
1. CESK기계는 메모리 주소를 다룰 수 있으므로 가비지 컬렉션(garbage collection, \`쓰레기 수집\'으로도 옮김)과 같은 자동 메모리 관리 방식도 추상적으로 표현할 수 있다. CESK기계에서 가비지 컬렉션을 어떻게 나타낼 수 있을지 생각해 보고 관련 문헌도 조사해 보라.